# Contains Neural Network and IID Cross-Validation

In [15]:
#calculate Baseline Score!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [16]:
import numpy as np
import pandas as pd
import copy

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit

#just for testing:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

In [17]:
#reading dataframe
df = pd.read_csv("dataset/phase_3_TRAIN_7d499bff69ca69b6_6372c3e_MLPC2021_generic.csv")

In [18]:
# drop target value, student annotations and string ID from input features:
X = df.drop(columns=['quadrant','mean_A','mean_V','id','score_mode','score_key_strength'])

# we want to predict the quadrant:
y = df['quadrant'].values

In [19]:
#splitting by pianist and piecw!

#create tags_dataframe:
X_tags=pd.DataFrame()
X_tags['id']=df['id']


#extract piece_id and pianist to later allow by piece/pianist/both cross validation
def extractPianist(x):
    return x[0:2]
def extract_piece_id(x):
    return x[3:5]
def extract_snippet_number(x):
    return x[6:9]

X_tags['Pianist']=X_tags['id'].apply(extractPianist)
X_tags['Piece_id']=X_tags['id'].apply(extract_piece_id)
X_tags['Snippet_number']=X_tags['id'].apply(extract_snippet_number)
X_tags['class']=df['quadrant']

X_tags.head()

,id,Pianist,Piece_id,Snippet_number,class
0,GG-01-000,GG,01,000,1
1,GG-01-001,GG,01,001,1
2,GG-01-002,GG,01,002,1
3,GG-01-003,GG,01,003,1
4,GG-01-004,GG,01,004,1


In [20]:
#get list of pianists and pieces!
pianist_list=list(set(X_tags['Pianist']))
piece_list=list(set(X_tags['Piece_id']))
len(piece_list)

36

In [21]:
#get distribution of classes per pianist
pianist_dist_list=[]
pianist_dist_list_perecent=[]
for pianist in pianist_list:  
    mylist=[]
    for i in range(4):
        mylist.append(len(X_tags.loc[(X_tags['Pianist'] == pianist) & (X_tags['class']==i+1)]))
    pianist_dist_list.append(mylist)
    mylist=[element/sum(mylist) for element in mylist]
    pianist_dist_list_perecent.append(mylist)

In [22]:
pianist_dist_list

[[70, 46, 294, 72],
 [73, 48, 210, 95],
 [95, 62, 148, 65],
 [71, 58, 183, 70],
 [70, 28, 280, 83],
 [94, 32, 265, 95]]

In [23]:
#distribution of classes per pianist seems to be fairly uniform accross all pianist: ->reandom cv_split!

In [24]:
#choose random pianists
pianist_cv=[]
number_folds=3

#initialize pianists_cv:
for i in range(number_folds):
    pianist_cv.append([])
    
#assign pianists to folds randomly
cp_pianist_list=pianist_list.copy()
for i in range(len(cp_pianist_list)):
    choosen_pianist=cp_pianist_list[np.random.randint(len(cp_pianist_list), size=1)[0]]
    cp_pianist_list.remove(choosen_pianist)
    pianist_cv[len(cp_pianist_list)%number_folds].append(choosen_pianist)

In [25]:
pianist_cv

[['GG', 'FG'], ['RT', 'AS'], ['SR', 'AH']]

In [26]:
#get distribution of classes per piece
piece_dist_list=[]
piece_dist_list_perecent=[]
for piece in piece_list:  
    mylist=[]
    for i in range(4):
        mylist.append(len(X_tags.loc[(X_tags['Piece_id'] == piece) & (X_tags['class']==i+1)]))
    piece_dist_list.append(mylist)
    mylist=[element/sum(mylist) for element in mylist]
    piece_dist_list_perecent.append(mylist)
    

In [27]:
#piece_dist_list 
# shows that a lot of pieces are only labled as one emotion! To ensure that all 4 emotions a present
# in the test set we can not do a completly random split! IDEA: Create 4 Subesets such that in each subset the majority class of 
# every piece is the same and then assign sumples from each subset randomly and uiformly to the train test splits

In [30]:
#create 4 subsets:
piece_subsets=[]

#initialize piece_cv:
for i in range(4):
    piece_subsets.append([])
piece_list_cp=copy.deepcopy(piece_list)
for p,piece in enumerate(piece_list):
    done=False
    for i in range(4):
        if not done:
            #print(piece_dist_list[p][i])
            #print(max(piece_dist_list[p]))
            if piece_dist_list[p][i]==max(piece_dist_list[p][:]):
                piece_subsets[i].append(piece_list[p])
                piece_list_cp.remove
                done=True

In [31]:
#just double checking!
np.shape(piece_subsets)
piece_subsets
mysum=0
for i in range(4):
    mysum+=len(piece_subsets[i])
mysum

36

In [32]:
# chose random pieces using above subset technique:
piece_cv=[]
number_folds=5
#initialize piece_cv:
for i in range(number_folds):
    piece_cv.append([])
    
cp_pi_subs = copy.deepcopy(piece_subsets)
#assign pieces to folds randomly
for i in range(4):
    fold=0
    for p in range(len(piece_subsets[i])):
        chos_piece=cp_pi_subs[i][np.random.randint(len(cp_pi_subs[i]), size=1)[0]]
        cp_pi_subs[i].remove(chos_piece)
        piece_cv[fold].append(chos_piece)
        if fold>=number_folds-1:
            fold=0
        else:
            fold+=1


In [33]:
piece_cv

[['27', '41', '39', '04', '15', '35', '16', '46', '45'],
 ['37', '29', '11', '47', '08', '02', '34'],
 ['30', '21', '20', '32', '28', '25', '26'],
 ['05', '18', '03', '12', '24', '01', '22'],
 ['09', '14', '31', '36', '19', '38']]

In [34]:
#just double checking!
np.shape(piece_cv)
piece_subsets
mysum=0
for i in range(len(piece_cv)):
    mysum+=len(piece_cv[i])
mysum

36

In [ ]:
#  mask=np.where((X_tags['Pianist'] == pianist_cv[f][i]))[0].index.values.astype(int)

In [103]:
#creating masks for gridsearch cv:

#create vertical splits:
myCViterator = []
for f,fold in enumerate(pianist_cv):
    vetrical_mask_train=np.zeros(0).astype(int)
    vetrical_mask_test=np.zeros(0).astype(int)
    for i in range(len(fold)):
        mask= X_tags[X_tags['Pianist']!=pianist_cv[f][i]].index.values.astype(int)
        vetrical_mask_train=np.concatenate((vetrical_mask_train,mask), axis=0)
        
        mask= X_tags[X_tags['Pianist']==pianist_cv[f][i]].index.values.astype(int)
        vetrical_mask_test=np.concatenate((vetrical_mask_test,mask), axis=0)
    #print(vetrical_mask_test)
    print(vetrical_mask_train)
    trainIndices=vetrical_mask_train
    testIndices=vetrical_mask_test
    myCViterator.append((trainIndices, testIndices))

#and create horizontal splits:
for f,fold in enumerate(piece_cv):
    vetrical_mask_train=np.zeros(0).astype(int)
    vetrical_mask_test=np.zeros(0).astype(int)
    for i in range(len(fold)):
        mask= X_tags[X_tags['Piece_id']!=piece_cv[f][i]].index.values.astype(int)
        vetrical_mask_train=np.concatenate((vetrical_mask_train,mask), axis=0)
        
        mask= X_tags[X_tags['Piece_id']==piece_cv[f][i]].index.values.astype(int)
        vetrical_mask_test=np.concatenate((vetrical_mask_test,mask), axis=0)
    #print(vetrical_mask_test)
    print(vetrical_mask_train)
    trainIndices=vetrical_mask_train
    testIndices=vetrical_mask_test
    myCViterator.append((trainIndices, testIndices))

myCViterator

[ 370  371  372 ... 2604 2605 2606]
[   0    1    2 ... 2604 2605 2606]
[   0    1    2 ... 2604 2605 2606]
[   0    1    2 ... 2604 2605 2606]
[   0    1    2 ... 2604 2605 2606]
[   0    1    2 ... 2604 2605 2606]
[   0    1    2 ... 2604 2605 2606]
[   0    1    2 ... 2604 2605 2606]


"\nfor i in range(nFolds):\n    trainIndices = myDf[ myDf['cvLabel']!=i ].index.values.astype(int)\n    testIndices =  myDf[ myDf['cvLabel']==i ].index.values.astype(int)\n    myCViterator.append( (trainIndices, testIndices) )\nvetrical_mask=[[2,3,4],[3,4,54]]\n"

In [104]:
gs_cv = GridSearchCV(MLPClassifier(hidden_layer_sizes=(4,40)), param_grid = {}, cv=myCViterator)
gs_cv.fit(pd.DataFrame(X), y)

GridSearchCV(cv=[(array([ 370,  371,  372, ..., 2604, 2605, 2606]),
                  array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  9...
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=200, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_state=None, shuffle=True,
                

In [105]:
gs_cv.cv_results_

{'mean_fit_time': array([18.95722851]),
 'std_fit_time': array([18.49769549]),
 'mean_score_time': array([0.0117979]),
 'std_score_time': array([0.0042232]),
 'params': [{}],
 'split0_test_score': array([0.52816901]),
 'split1_test_score': array([0.51612903]),
 'split2_test_score': array([0.5524239]),
 'split3_test_score': array([0.65987461]),
 'split4_test_score': array([0.57491857]),
 'split5_test_score': array([0.41386555]),
 'split6_test_score': array([0.63326226]),
 'split7_test_score': array([0.65609756]),
 'mean_test_score': array([0.56684256]),
 'std_test_score': array([0.07808296]),
 'rank_test_score': array([1])}